## Se planean crear 3 tipos de modelos, comparar y luego elegir el mejor

## 1. Random Forest

### imports

In [1]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

In [2]:
# read the data
df_mean = pd.read_csv('assets/cleaned_train.csv')

In [3]:
#df_mean.info()
df_mean.head()

,Id,minutes_past,radardist_km,Ref,Ref_5x5_10th,Ref_5x5_50th,Ref_5x5_90th,RefComposite,RefComposite_5x5_10th,RefComposite_5x5_50th,...,RhoHV_5x5_90th,Zdr,Zdr_5x5_10th,Zdr_5x5_50th,Zdr_5x5_90th,Kdp,Kdp_5x5_10th,Kdp_5x5_50th,Kdp_5x5_90th,Expected
0,1,29.833333,10.0,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.254000
1,2,29.083333,2.0,16.625,13.666667,17.375,21.333333,22.666667,20.375000,22.916667,...,1.000000,0.380208,0.119792,0.416667,0.781250,-0.288187,-1.448893,-0.319096,1.116661,1.016001
2,3,30.750000,10.0,0.000,0.000000,0.000,8.500000,0.000000,0.000000,0.000000,...,0.801667,0.000000,0.000000,0.000000,2.062500,0.000000,0.000000,0.000000,0.000000,26.162014
3,4,28.153846,9.0,26.600,20.071429,25.800,30.269231,26.666667,21.090909,25.115385,...,1.015833,-1.125000,0.000000,0.500000,1.515625,7.029999,0.000000,0.000000,6.329994,4.064002
4,5,28.714286,13.0,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,774.700440


In [4]:
# Set data frames to be the inputs of the skit model
X = df_mean.iloc[:, :-1]
Y = df_mean.iloc[:, 23:]
Y = Y.values.ravel()

In [5]:
#   Scaling data frame
# scaling X
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)

In [6]:
# splits data

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)


In [7]:
# Fit the model

clf = RandomForestRegressor(random_state=42, max_depth=35, n_jobs=-1)
clf.fit(X_train, Y_train)

In [ ]:
# Get predictions and evaluate performance
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Predecir valores usando el modelo entrenado
y_pred = clf.predict(X_test)

# Calcular métricas de regresión
mse = mean_squared_error(Y_test, y_pred)
mae = mean_absolute_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

# Imprimir resultados
# MSE: Un MSE bajo indica que el modelo predice los valores reales con mayor precisión.
print("Mean Squared Error (MSE):", mse)
# MAE:  Un MAE bajo indica que el error promedio entre los valores predichos y reales es pequeño.
# En general, se busca un MAE lo más cercano a 0 posible. 
print("Mean Absolute Error (MAE):", mae)
# R-squared: Un valor de R-cuadrado cercano a 1 
# indica que el modelo explica una gran proporción de la variabilidad en los datos.
print("R-squared:", r2)

rfr_m = [mse, mae, r2]

NameError: name 'clf' is not defined

## Neural Network


In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

In [ ]:
# read the data
df_mean = pd.read_csv('assets/cleaned_train.csv')

In [ ]:
# Set data frames to be the inputs of the skit model
X = df_mean.iloc[:, :-1]
Y = df_mean.iloc[:, 23:]

In [ ]:
#   Scaling data frame
# scaling X
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = sc.fit_transform(X)

In [ ]:
# splits data

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [ ]:
# Ensure Y_train is a 2D array
Y_train = np.reshape(Y_train, (-1, 1))

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)  # Use the same scaler for test data

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
# fit

ann = Sequential()
ann.add( Dense(64, input_shape=(X_train.shape[1],), activation='softmax') )
ann.add( Dense(64, activation='softmax'))
ann.add( Dense(64, activation='softmax'))
ann.add( Dense(64, activation='softmax'))
ann.add( Dense(64, activation='softmax'))
ann.add(Dense(1, activation='sigmoid'))

# compile the ann
ann.compile(optimizer=Adam(learning_rate=1e-3), 
            loss='mean_squared_error', 
            metrics=[BinaryAccuracy(), FalseNegatives(),],)

In [ ]:
print(X_train.shape, len(X_train), type(X_train))
print(Y_train.shape, len(Y_train), type(Y_train))

(1029447, 23) 1029447 <class 'numpy.ndarray'>
(1029447, 1) 1029447 <class 'numpy.ndarray'>


In [ ]:
# train ann model
ann.fit(X_train, Y_train, epochs=100, batch_size=1000, verbose=-1)



Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Compile the model
ann.compile(optimizer='adam', loss='mean_squared_error', metrics=[MeanSquaredError(), MeanAbsoluteError()])

# Evaluate the model
y_pred = ann.predict(X_test)

# Calculate metrics
mse = mean_squared_error(Y_test, y_pred)
mae = mean_absolute_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

# Print results
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared:", r2)
ann_m = [mse, mae, r2]

3575/3575 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step
Mean Squared Error (MSE): 31222.500208771537
Mean Absolute Error (MAE): 62.26429606489105
R-squared: -9.429454803466797e-05


##compare models

In [ ]:
print(f'Random forest: {rfr_m}')